<a href="https://colab.research.google.com/github/danielk1345/ML-Alpaca-ImgRecog/blob/main/Data_Loading_AIPaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
DATADIR = '/content/gdrive/MyDrive/AIpaca'
CATEGORIES = ["alpaca", "not alpaca"]


Mounted at /content/gdrive


In [ ]:
training_data = []
IMG_SIZE = 150
def create_training_data():
  IMG_SIZE = 150
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    classification_num = CATEGORIES.index(category)
    # 0 for an alpaca, 1 for not an alpaca
    for img in os.listdir(path):
      try:
        img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, classification_num])
      except Exception as e:
        pass

create_training_data()

In [ ]:
print(len(training_data))

659


In [ ]:
import random
random.shuffle(training_data)

In [ ]:
X = []
y = []

In [ ]:
for feature,label in training_data:
  X.append(feature)
  y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE,1)


In [ ]:
import pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X,pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X/255.0


model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy",
             optimizer="Adam",
             metrics=["accuracy"])
y = np.array(y)

model.fit(X, y, batch_size=32, epochs = 32, validation_split=0.2)

Epoch 1/32
17/17 [==============================] - 28s 2s/step - loss: 2.5927 - accuracy: 0.5522 - val_loss: 0.6272 - val_accuracy: 0.6742
Epoch 2/32
17/17 [==============================] - 27s 2s/step - loss: 0.6539 - accuracy: 0.5939 - val_loss: 0.6080 - val_accuracy: 0.6970
Epoch 3/32
17/17 [==============================] - 28s 2s/step - loss: 0.5869 - accuracy: 0.7021 - val_loss: 0.6322 - val_accuracy: 0.7045
Epoch 4/32
17/17 [==============================] - 26s 2s/step - loss: 0.5002 - accuracy: 0.7590 - val_loss: 0.5944 - val_accuracy: 0.7500
Epoch 5/32
17/17 [==============================] - 26s 2s/step - loss: 0.3865 - accuracy: 0.8292 - val_loss: 0.7932 - val_accuracy: 0.5758
Epoch 6/32
17/17 [==============================] - 26s 2s/step - loss: 0.3213 - accuracy: 0.8861 - val_loss: 0.6023 - val_accuracy: 0.7273
Epoch 7/32
17/17 [==============================] - 26s 2s/step - loss: 0.2418 - accuracy: 0.9146 - val_loss: 0.7172 - val_accuracy: 0.6591
Epoch 8/32
17/17 [==

In [ ]:
model.save('Alpaca_CNN.model')

INFO:tensorflow:Assets written to: Alpaca_CNN.model/assets
